In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import glob, time, pickle, os, sys
from scipy.stats import chi2
import astropy.units as u

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter, gp_signals, utils, signal_base

import enterprise_extensions as e_e
from enterprise_extensions import model_utils, blocks, deterministic
from enterprise_extensions.frequentist import F_statistic

In [5]:
current_path = os.getcwd()
splt_path = current_path.split("/")
top_path_idx = splt_path.index("akaiser")
#top_path_idx = splt_path.index("nanograv")
#top_path_idx = splt_path.index("ark0015")
top_dir = "/".join(splt_path[0 : top_path_idx + 1])

noise_path = top_dir + "/pta_sim/pta_sim"
sys.path.insert(0, noise_path)
import noise

In [6]:
datarelease = '11yr'
save_dir = './results_v2/{}'.format(datarelease)
psr_obj_file = './eleven_yr_psr_objects_v4.pickle'
load_from_file = False
save_to_file = True
save_version = '3_svsetup'
n_freq_max = 1

datadir = top_dir + "/{}".format(datarelease)
parfiles = sorted(glob.glob(datadir + "/par/*.par"))
timfiles = sorted(glob.glob(datadir + "/tim/*.tim"))
noisefiles = sorted(glob.glob(datadir + "/noisefiles/*.txt"))

eleven_yr_psr_list = datadir + '/psrlist_Tg3yr.txt'
eleven_yr_psrs = [line.rstrip('\n') for line in open(eleven_yr_psr_list,'r')]

In [7]:
for parfile in parfiles:
    if 'J1713+0747' in parfile.split('/')[-1].split('_'):
        if 't2' in parfile.split('/')[-1].split('.'):
            parfiles.remove(parfile)

In [8]:
noisedict = {}
for noisefile in noisefiles:
    tmpnoisedict = {}
    tmpnoisedict = noise.get_noise_from_file(noisefile)
    for og_key in tmpnoisedict.keys():
        #split_key = og_key.split('_')
        #psr_name = split_key[0]
        #param = "_".join(split_key[1:])
        #new_key = "_".join([psr_name,"_".join(param.split("-"))])
        if og_key.split('_')[0] in eleven_yr_psrs:
            noisedict[og_key] = tmpnoisedict[og_key]

In [9]:
noisedict_gp_ecorr = {}
for key,value in noisedict.items():
    if '-' in str(key.split('_')[1:]):
        new_key = key.split('_')[0] + '_' + '_'.join('_'.join(key.split('_')[1:]).split('-'))
    else:
        new_key = key
        
    if 'ecorr' in new_key.split('_'):
        
        split_key = new_key.split('_')
        #new_key = '_'.join(split_key[:-3]) + '_basis_ecorr_' + '_'.join(split_key[-2:])
        #new_key = '_'.join(split_key[:-3]) + '_basis_ecorr'
        new_key = split_key[0] + '_basis_ecorr_' + '_'.join(split_key[1:])
        
    noisedict_gp_ecorr[new_key] = value

In [14]:
for key, val in noisedict.items():
    if key.split('_')[0] == 'B1855+09':
        print(key,': ',val)
#print(noisedict.keys())

B1855+09_430_PUPPI_efac :  1.11896
B1855+09_L-wide_PUPPI_efac :  1.38104
B1855+09_430_ASP_efac :  1.16587
B1855+09_L-wide_ASP_efac :  1.08538
B1855+09_430_ASP_log10_ecorr :  -8.47348
B1855+09_430_PUPPI_log10_ecorr :  -6.31096
B1855+09_L-wide_ASP_log10_ecorr :  -6.09208
B1855+09_L-wide_PUPPI_log10_ecorr :  -6.401
B1855+09_430_PUPPI_log10_equad :  -6.17415
B1855+09_L-wide_PUPPI_log10_equad :  -6.53715
B1855+09_430_ASP_log10_equad :  -7.93502
B1855+09_L-wide_ASP_log10_equad :  -6.51038
B1855+09_red_noise_log10_A :  -13.8022
B1855+09_red_noise_gamma :  3.63368
